In [3]:
from qick import *

# the main program class
from qick.asm_v2 import AveragerProgramV2
# for defining sweeps
from qick.asm_v2 import QickSpan, QickSweep1D
from qick.asm_v2 import QickProgramV2, AcquireProgramV2

# jupyter setup boilerplate
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger

import time
import matplotlib.pyplot as plt


soc = QickSoc('qick_216_test.bit')
#soccfg = soc
#soc.tproc.info()


In [27]:
print('Measuring Clock Cycles Between READS')
Test_qty  = 10
Time_test  = np.zeros(Test_qty, dtype=np.int32)

asm = """
REG_WR s12  imm #0
INIT:
   REG_WR s12  op -op(s12 + #1)
   JUMP INIT
"""

p_txt, p_bin  = Assembler.str_asm2bin(asm)
soc.tproc.Load_PMEM(p_bin)
#for val in p_txt:
#    print (val)

soc.tproc.read_sel  = 1

start = time.time()
soc.tproc.start()
for i in range(0,Test_qty):
    Time_test[i] = soc.tproc.tproc_r_dt1

soc.tproc.stop()
stop = time.time()

print(Time_test)

print('Test Time Elapsed: ',stop-start)
print('Time between Flag Change Max >',np.max(Test_var))
print('Time between Flag Change Min >',np.min(Test_var))
#plt.hist(Test_var, histtype='step', bins=100)

a = np.histogram(Test_var)
print(a[0], sum(a[0]))




Measuring Clock Cycles Between READS
[23428 24676 25536 26347 27142 27945 28732 29524 30317 31112]
Test Time Elapsed:  0.0008683204650878906
Time between Flag Change Max > 33888
Time between Flag Change Min > 26148
[1 1 1 1 1 1 1 1 1 1] 10


In [25]:
print('Measuring FLAG write TIME> reading s13 and s14')
Test_qty  = 10
Time_set = np.zeros(Test_qty, dtype=np.int32)
Time_clr = np.zeros(Test_qty, dtype=np.int32)
Test_val  = np.zeros(Test_qty, dtype=np.int32)

asm = """
REG_WR s_cfg imm cfg_flg_axi

INIT:
   REG_WR s12  op -op(s12 + #1)
   REG_WR s13  imm #-1 -if(F)
   REG_WR s13  imm #2  -if(NF)
   JUMP INIT
"""

p_txt, p_bin  = Assembler.str_asm2bin(asm)
soc.tproc.Load_PMEM(p_bin)
#for val in p_txt:
#    print (val)

soc.tproc.read_sel  = 1

start = time.time()
soc.tproc.start()
for i in range(0,Test_qty):
#    soc.tproc.set_axi_flg()
#    Time_set[i] = soc.tproc.tproc_r_dt1
#    soc.tproc.clr_axi_flg()
    Time_clr[i] = soc.tproc.tproc_r_dt1

soc.tproc.stop()
stop = time.time()

Test_var = Time_clr - Time_set
print(Time_set)
print(Time_clr)

print('Test Time Elapsed: ',stop-start)
print('Time between Flag Change Max >',np.max(Test_var))
print('Time between Flag Change Min >',np.min(Test_var))
#plt.hist(Test_var, histtype='step', bins=100)

a = np.histogram(Test_var)
print(a[0], sum(a[0]))




Measuring FLAG write TIME> reading s13 and s14
[0 0 0 0 0 0 0 0 0 0]
[15649 16431 17021 17566 18101 18642 19180 19711 20249 20782]
Test Time Elapsed:  0.0008676052093505859
Time between Flag Change Max > 20782
Time between Flag Change Min > 15649
[1 1 1 1 1 1 1 1 1 1] 10


In [ ]:
print('Measuring FLAG write TIME> Using Memory')
Test_qty  = 200
Time_set = np.zeros(Test_qty, dtype=np.int32)
Time_clr = np.zeros(Test_qty, dtype=np.int32)
Test_val  = np.zeros(Test_qty, dtype=np.int32)

asm = """
.ALIAS mem_addr r15
REG_WR s_cfg imm cfg_flg_axi

INIT:
   REG_WR s12  op -op(s12 + #1)
   DMEM_WR [mem_addr]
   DMEM_WR [mem_addr + &1]
   REG_WR s13  imm #-1 -if(F)
   REG_WR s13  imm #2  -if(NF)
   JUMP INIT
"""

p_txt, p_bin  = Assembler.str_asm2bin(asm)
soc.tproc.Load_PMEM(p_bin)
#for val in p_txt:
#    print (val)

soc.tproc.read_sel  = 1

start = time.time()
soc.tproc.start()
for i in range(0,Test_qty,2):
    soc.tproc.set_axi_flg()
    soc.tproc.clr_axi_flg()

soc.tproc.stop()
stop = time.time()

Test_var = Time_clr - Time_set

print('Test Time Elapsed: ',stop-start)
print('Time between Flag Change Max >',np.max(Test_var))
print('Time between Flag Change Min >',np.min(Test_var))
#plt.hist(Test_var, histtype='step', bins=100)

a = np.histogram(Test_var)
print(a[0], sum(a[0]))


